<a href="https://colab.research.google.com/github/mingmcs/pyhealth/blob/week6/Tutorial_4_pyhealth_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Preparation**
- install pyhealth alpha version 

In [ ]:
!pip install pyhealth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 24.8 MB/s eta 0:00:00


### **Instruction on [pyhealth.trainer.Trainer](https://pyhealth.readthedocs.io/en/latest/api/trainer.html)**
- **[README]**: The Trainer class is the training handler (similar to [pytorch-lightning.Trainer](https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html)) in the package. We use it to train the ML and DL model. It has the following arguments and functionality

- **[Arguments]**: 
To initialize a trainer instance, the following environments should be specified.
  - `model`: the pyhealth.models object
  - `checkpoint_path`: path to intermediate checkpoint
  - `metric`: which metrics to record during training. For example, we can record the pr_auc and auc_roc metrics. 
  - `device`: device to use
  - `enable_logging`: enable logging
  - `output_path`: output path
  - `exp_name`: experiment/task name

- **[Functionality]**:
  - `Trainer.train()`: simply call the `.train()` function will start to train the DL or ML model.
    - `train_dataloader`: train data loader
    - `val_dataloader`: validation data loader
    - `epochs`: number of epochs to train the model
    - `optimizer_class`: optimizer, such as `torch.optim.Adam`
    - `optimizer_params`: optimizer parameters, including
      - `lr`: learning rate
      - `weight_decay`: weight decay
    - `max_grad_norm`: max gradient norm
    - `monitor`: metric name to monitor, default is None
    - `monitor_criterion`: criterion to monitor, default is "max"
    - `load_best_moel_at_last`: whether to load the best model during the last iteration.

### **Step 1 & 2 & 3: Prepare datasets, task, and model**
- Example: We use **MIMIC-III dataset** and **RETAIN** model for **readmission prediction** task. Refer to [Tutorial 1](https://colab.research.google.com/drive/18kbzEQAj1FMs_J9rTGX8eCoxnWdx4Ltn?usp=sharing), [Tutorial 2](https://colab.research.google.com/drive/1r7MYQR_5yCJGpK_9I9-A10HmpupZuIN-?usp=sharing), and [Tutorial 3](https://colab.research.google.com/drive/1LcXZlu7ZUuqepf269X3FhXuhHeRvaJX5?usp=sharing).

In [ ]:
# load dataset
from pyhealth.datasets import MIMIC3Dataset
mimic3dataset = MIMIC3Dataset(
    root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
    tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
    dev=False, 
)


Mapping codes: 100%|██████████| 49993/49993 [00:01<00:00, 38089.70it/s]


In [ ]:
from pyhealth.tasks import readmission_prediction_mimic3_fn
from pyhealth.datasets import split_by_patient, get_dataloader

# set task
dataset = mimic3dataset.set_task(task_fn=readmission_prediction_mimic3_fn)

# dataset split
train_ds, val_ds, test_ds = split_by_patient(dataset, [0.8, 0.1, 0.1])

# obtain train/val/test dataloader, they are <torch.data.DataLoader> object
train_loader = get_dataloader(train_ds, batch_size=32, shuffle=True)
val_loader = get_dataloader(val_ds, batch_size=32, shuffle=False)
test_loader = get_dataloader(test_ds, batch_size=32, shuffle=False)

Generating samples for readmission_prediction_mimic3_fn: 100%|██████████| 49993/49993 [00:00<00:00, 191070.25it/s]


In [ ]:
# use RETAIN model
from pyhealth.models import RETAIN

model = RETAIN(
    dataset=dataset,
    # look up what are available for "feature_keys" and "label_keys" in dataset.samples[0]
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)

### **How to use the Trainer**
- We first initialize the trainer by config the environments, setting `enable_logging` to be True, assign the output_path as `"../output"`, and specify the device.
- We use the trainer for training the `model`. In this step, we need the `train_loader` for model training, `val_loader` to do hold-out validation and use `average_precision_score` as the monitoring metric to select the best model. Training epoch is set 5.

In [ ]:
from pyhealth.trainer import Trainer

trainer = Trainer(model=model)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=50,
    monitor="roc_auc",
)

RETAIN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (retain): ModuleDict(
    (conditions): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
    (procedures): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)
Metrics: None
Device: cpu

Training:
Batch size: 32
Optimizer: <class 't

Epoch 0 / 50:   0%|          | 0/55 [00:00<?, ?it/s]

--- Train epoch-0, step-55 ---
loss: 0.4712
Evaluation: 100%|██████████| 7/7 [00:00<00:00, 62.86it/s]
--- Eval epoch-0, step-55 ---
pr_auc: 0.1755
roc_auc: 0.6045
f1: 0.0000
loss: 0.3530
New best roc_auc score (0.6045) at epoch-0, step-55



Epoch 1 / 50:   0%|          | 0/55 [00:00<?, ?it/s]

--- Train epoch-1, step-110 ---
loss: 0.3831
Evaluation: 100%|██████████| 7/7 [00:00<00:00, 80.47it/s]
--- Eval epoch-1, step-110 ---
pr_auc: 0.1541
roc_auc: 0.6276
f1: 0.0000
loss: 0.3483
New best roc_auc score (0.6276) at epoch-1, step-110



...

--- Train epoch-47, step-2640 ---
loss: 0.0273
Evaluation: 100%|██████████| 7/7 [00:00<00:00, 124.30it/s]
--- Eval epoch-47, step-2640 ---
pr_auc: 0.1377
roc_auc: 0.5962
f1: 0.0645
loss: 0.6133



Epoch 48 / 50:   0%|          | 0/55 [00:00<?, ?it/s]

--- Train epoch-48, step-2695 ---
loss: 0.0388
Evaluation: 100%|██████████| 7/7 [00:00<00:00, 142.49it/s]
--- Eval epoch-48, step-2695 ---
pr_auc: 0.1456
roc_auc: 0.6019
f1: 0.0645
loss: 0.6024



Epoch 49 / 50:   0%|          | 0/55 [00:00<?, ?it/s]

--- Train epoch-49, step-2750 ---
loss: 0.0333
Evaluation: 100%|██████████| 7/7 [00:00<00:00, 139.44it/s]
--- Eval epoch-49, step-2750 ---
pr_auc: 0.1467
roc_auc: 0.6159
f1: 0.0741
loss: 0.6072
Loaded best model


### **Evaluation**

In [ ]:
# evaluation option 1: use our built-in evaluation metric
result = trainer.evaluate(test_loader)
print ('\n', result)

# evaluation option 2: use pyhealth.metrics
from pyhealth.metrics.binary import binary_metrics_fn
y_true, y_prob, loss = trainer.inference(test_loader)
result = binary_metrics_fn(y_true, y_prob, metrics=["pr_auc", "roc_auc"])
print ('\n', result)

# evaluation option 3: use sklearn.metrics
from sklearn.metrics import average_precision_score, roc_auc_score
y_pred = (y_prob > 0.5).astype('int')
print (
    '\n',
    'roc_auc', roc_auc_score(y_true, y_prob), 
    'pr_auc:', average_precision_score(y_true, y_prob)
)

Evaluation: 100%|██████████| 7/7 [00:00<00:00, 131.55it/s]



 {'pr_auc': 0.1511461740187427, 'roc_auc': 0.5009398496240602, 'f1': 0.0, 'loss': 0.43073106024946484}


Evaluation: 100%|██████████| 7/7 [00:00<00:00, 125.87it/s]



 {'pr_auc': 0.1511461740187427, 'roc_auc': 0.5009398496240602}

 roc_auc 0.5009398496240602 pr_auc: 0.1511461740187427


If you find it useful, please give us a star ⭐ (fork, and watch) at https://github.com/sunlabuiuc/PyHealth. 

Thanks very much for your support!